In [1]:
from document_extraction import DoclingExtractor

In [2]:

source =r"C:\Users\tallar\Downloads\2025+AI+Barometer+-+Sailpeak+-+Fintech+Belgium+vDIGITAL+(1).pdf"
source = r"C:\Users\tallar\Documents\PROJETS\GenAI\docs\files\test_tel.pdf"
source = r"C:\Users\tallar\Downloads\Famille bamendjou de liege-20250801T201343Z-1-001\Famille bamendjou de liege\ARESBAL_ ROI et statut.pdf"
source=r"C:\Users\tallar\Documents\PROJETS\GenAI\docs\files\compil eguichet.pdf"

In [3]:
print(source)

C:\Users\tallar\Documents\PROJETS\GenAI\docs\files\compil eguichet.pdf


In [4]:
dox = DoclingExtractor(source)
doc=dox.run()

le document docling extraact C:\Users\tallar\Documents\PROJETS\GenAI\docs\files\compil eguichet.pdf


C:\Users\tallar\Documents\PROJETS\GenAI\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Parameter `strict_text` has been deprecated and will be ignored.


In [5]:
res = dox.docling_extraction(source)
spl = res.split('\n\n')

le document docling extraact C:\Users\tallar\Documents\PROJETS\GenAI\docs\files\compil eguichet.pdf


Parameter `strict_text` has been deprecated and will be ignored.


In [11]:
print(dox.get_chunks())

AttributeError: 'DoclingExtractor' object has no attribute 'get_chunks'

In [7]:

res_psl = [p.strip().replace('\n', ' ') for p in spl if len(p.strip()) > 40]
print(res_psl)

['Concernant le dossier 556-1573, Géhénot Igor Henri affirme avoir payé 100 EUR pour la créance 59092105040 au Schmit Isabelle à la date du 22/09/2023.  Géhénot Igor Henri est domicilié à Rue Vanderkindere 366, 2 1180 Uccle et a pour adresse mail : igor.g@live.be.', 'Concernant le dossier 556-1681, Diallo Moussa affirme avoir payé 108,04 EUR pour la créance 0207343933  au à la date du 06/01/2022.  Diallo Moussa est domicilié à Rue Général-de-Gaulle 74, 0053 4020 Liège et a pour adresse mail : diallo.moussa86@gmail.com.', 'Concernant le dossier 556-1702, peeters didier affirme avoir payé 23565,75 EUR pour la créance 20205080000212  au   à la date du 10/03/2021.  peeters didier est domicilié à chaussée de Waterloo 1359, 4 1180 Uccle et a pour adresse mail : didier.peeters@cdp.be.', 'Concernant le dossier 556-2851, Paulus Théodore affirme avoir payé 1303,26 EUR pour la créance 20205150001270  au   à la date du 17/01/2022.  Paulus Théodore est domicilié à Rue defechereux 17,  4031 Angleur 

In [8]:
print(len(doc))

90


In [9]:
print(DoclingExtractor(source).run())

le document docling extraact C:\Users\tallar\Documents\PROJETS\GenAI\docs\files\compil eguichet.pdf


KeyboardInterrupt: 